In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [9]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [10]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [11]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                                    
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()
                    

In [12]:
cap.release()
cv2.destroyAllWindows()

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
np.array(sequences).shape

(90, 30, 1662)

In [18]:
np.array(labels).shape

(90,)

In [19]:
X = np.array(sequences)


In [20]:
X.shape

(90, 30, 1662)

In [21]:
y = to_categorical(labels).astype(int)

In [22]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [24]:
y_test.shape

(5, 3)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [26]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [27]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [28]:
res = [.7, 0.2, 0.1]

In [29]:
actions[np.argmax(res)]

'hello'

In [30]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 7s 145ms/step - loss: 6.4083 - categorical_accuracy: 0.4000
Epoch 2/2000
3/3 [==============================] - 0s 149ms/step - loss: 3.0134 - categorical_accuracy: 0.3412
Epoch 3/2000
3/3 [==============================] - 0s 150ms/step - loss: 2.3871 - categorical_accuracy: 0.2941
Epoch 4/2000
3/3 [==============================] - 1s 189ms/step - loss: 4.9170 - categorical_accuracy: 0.2706
Epoch 5/2000
3/3 [==============================] - 1s 222ms/step - loss: 2.6412 - categorical_accuracy: 0.4706
Epoch 6/2000
3/3 [==============================] - 1s 184ms/step - loss: 4.7127 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 160ms/step - loss: 8.8201 - categorical_accuracy: 0.2824
Epoch 8/2000
3/3 [==============================] - 1s 178ms/step - loss: 5.7064 - categorical_accuracy: 0.3294
Epoch 9/2000
3/3 [==============================] - 1s 190ms/step - loss: 2.7161 - categorical_accuracy:

3/3 [==============================] - 1s 190ms/step - loss: 0.9515 - categorical_accuracy: 0.5059
Epoch 74/2000
3/3 [==============================] - 1s 191ms/step - loss: 0.9892 - categorical_accuracy: 0.4824
Epoch 75/2000
3/3 [==============================] - 1s 169ms/step - loss: 0.9285 - categorical_accuracy: 0.4824
Epoch 76/2000
3/3 [==============================] - 1s 189ms/step - loss: 0.9748 - categorical_accuracy: 0.4941
Epoch 77/2000
3/3 [==============================] - 1s 180ms/step - loss: 0.9988 - categorical_accuracy: 0.3647
Epoch 78/2000
3/3 [==============================] - 1s 173ms/step - loss: 0.9659 - categorical_accuracy: 0.4706
Epoch 79/2000
3/3 [==============================] - 0s 166ms/step - loss: 1.0003 - categorical_accuracy: 0.4588
Epoch 80/2000
3/3 [==============================] - 1s 194ms/step - loss: 0.9812 - categorical_accuracy: 0.5765
Epoch 81/2000
3/3 [==============================] - 1s 176ms/step - loss: 1.0100 - categorical_accuracy: 0.45

3/3 [==============================] - 0s 140ms/step - loss: 0.8496 - categorical_accuracy: 0.5529
Epoch 146/2000
3/3 [==============================] - 0s 142ms/step - loss: 0.9110 - categorical_accuracy: 0.4471
Epoch 147/2000
3/3 [==============================] - 0s 143ms/step - loss: 0.9184 - categorical_accuracy: 0.6118
Epoch 148/2000
3/3 [==============================] - 0s 139ms/step - loss: 0.8626 - categorical_accuracy: 0.5176
Epoch 149/2000
3/3 [==============================] - 0s 149ms/step - loss: 0.8390 - categorical_accuracy: 0.7059
Epoch 150/2000
3/3 [==============================] - 0s 138ms/step - loss: 0.8990 - categorical_accuracy: 0.5176
Epoch 151/2000
3/3 [==============================] - 0s 134ms/step - loss: 0.7916 - categorical_accuracy: 0.6588
Epoch 152/2000
3/3 [==============================] - 0s 132ms/step - loss: 0.8672 - categorical_accuracy: 0.5294
Epoch 153/2000
3/3 [==============================] - 0s 129ms/step - loss: 1.2282 - categorical_accura

Epoch 217/2000
3/3 [==============================] - 1s 202ms/step - loss: 0.8449 - categorical_accuracy: 0.5765
Epoch 218/2000
3/3 [==============================] - 1s 187ms/step - loss: 0.8895 - categorical_accuracy: 0.4941
Epoch 219/2000
3/3 [==============================] - 1s 213ms/step - loss: 0.7861 - categorical_accuracy: 0.6471
Epoch 220/2000
3/3 [==============================] - 1s 208ms/step - loss: 0.7920 - categorical_accuracy: 0.6000
Epoch 221/2000
3/3 [==============================] - 1s 177ms/step - loss: 0.7806 - categorical_accuracy: 0.5647
Epoch 222/2000
3/3 [==============================] - 1s 197ms/step - loss: 0.7680 - categorical_accuracy: 0.6471
Epoch 223/2000
3/3 [==============================] - 1s 235ms/step - loss: 0.7707 - categorical_accuracy: 0.6471
Epoch 224/2000
3/3 [==============================] - 1s 191ms/step - loss: 0.7669 - categorical_accuracy: 0.6118
Epoch 225/2000
3/3 [==============================] - 1s 187ms/step - loss: 0.7491 - cat

3/3 [==============================] - 1s 185ms/step - loss: 0.7615 - categorical_accuracy: 0.6588
Epoch 289/2000
3/3 [==============================] - 1s 203ms/step - loss: 0.7828 - categorical_accuracy: 0.6000
Epoch 290/2000
3/3 [==============================] - 1s 179ms/step - loss: 0.7854 - categorical_accuracy: 0.5294
Epoch 291/2000
3/3 [==============================] - 0s 144ms/step - loss: 0.6843 - categorical_accuracy: 0.6706
Epoch 292/2000
3/3 [==============================] - 0s 153ms/step - loss: 0.7867 - categorical_accuracy: 0.6941
Epoch 293/2000
3/3 [==============================] - 1s 180ms/step - loss: 0.6339 - categorical_accuracy: 0.7176
Epoch 294/2000
3/3 [==============================] - 1s 204ms/step - loss: 0.7380 - categorical_accuracy: 0.6824
Epoch 295/2000
3/3 [==============================] - 1s 185ms/step - loss: 0.7213 - categorical_accuracy: 0.6941
Epoch 296/2000
3/3 [==============================] - 1s 194ms/step - loss: 0.7958 - categorical_accura

...

3/3 [==============================] - 0s 138ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1714/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1715/2000
3/3 [==============================] - 0s 155ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1716/2000
3/3 [==============================] - 0s 139ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1717/2000
3/3 [==============================] - 0s 138ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1718/2000
3/3 [==============================] - 0s 133ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1719/2000
3/3 [==============================] - 0s 148ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1720/2000
3/3 [==============================] - 0s 156ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1721/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.0981 - categorica

3/3 [==============================] - 1s 190ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1785/2000
3/3 [==============================] - 1s 159ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1786/2000
3/3 [==============================] - 1s 242ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1787/2000
3/3 [==============================] - 1s 208ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1788/2000
3/3 [==============================] - 1s 172ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1789/2000
3/3 [==============================] - 1s 253ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1790/2000
3/3 [==============================] - 1s 172ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1791/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1792/2000
3/3 [==============================] - 0s 138ms/step - loss: 1.0981 - categorica

3/3 [==============================] - 0s 143ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1856/2000
3/3 [==============================] - 0s 132ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1857/2000
3/3 [==============================] - 0s 142ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1858/2000
3/3 [==============================] - 1s 196ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1859/2000
3/3 [==============================] - 1s 178ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1860/2000
3/3 [==============================] - 1s 216ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1861/2000
3/3 [==============================] - 1s 182ms/step - loss: 1.0981 - categorical_accuracy: 0.2471
Epoch 1862/2000
3/3 [==============================] - 1s 180ms/step - loss: 1.0981 - categorical_accuracy: 0.2588
Epoch 1863/2000
3/3 [==============================] - 1s 169ms/step - loss: 1.0981 - categorica

3/3 [==============================] - 0s 125ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1927/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1928/2000
3/3 [==============================] - 0s 136ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1929/2000
3/3 [==============================] - 0s 126ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1930/2000
3/3 [==============================] - 0s 138ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1931/2000
3/3 [==============================] - 0s 134ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1932/2000
3/3 [==============================] - 0s 137ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1933/2000
3/3 [==============================] - 0s 134ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1934/2000
3/3 [==============================] - 0s 132ms/step - loss: 1.0981 - categorica

3/3 [==============================] - 0s 130ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1998/2000
3/3 [==============================] - 0s 136ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 1999/2000
3/3 [==============================] - 0s 135ms/step - loss: 1.0981 - categorical_accuracy: 0.3412
Epoch 2000/2000
3/3 [==============================] - 0s 132ms/step - loss: 1.0981 - categorical_accuracy: 0.3412


In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [33]:
res = model.predict(X_test)

1/1 [==============================] - 1s 797ms/step


In [34]:
actions[np.argmax(res[4])]

'hello'

In [35]:
actions[np.argmax(y_test[4])]

'iloveyou'

In [36]:
model.save('action.h5')

In [52]:
del model

In [53]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [54]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
model.load_weights('action.h5')

In [56]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [57]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 782ms/step


In [58]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [59]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 4],
        [0, 1]],

       [[4, 0],
        [1, 0]],

       [[2, 0],
        [3, 0]]], dtype=int64)

In [60]:
accuracy_score(ytrue, yhat)

0.2

In [74]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
#         sequence = sequence[:30]
#         sequence.append(keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
        #3. Viz logic
        if res[np.argmax(res)] > threshold or True: 
            print(actions[np.argmax(res)])
            if len(sentence) > 0: 
                if actions[np.argmax(res)] != sentence[-1] or True:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
        print(sentence)

        if len(sentence) > 5: 
            sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

hello
['hello']
hello
['hello', 'hello']
hello
['hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'hello', 'hello', 'hello']
hello
['hello', 'hello', 'hello', 'he